Velho, a ideia de forma mais detalhada é o seguinte:

1. Escolher um ativo alvo, que vamos tentar prever o preço dele na próxima semana ou mês. Não estou pensando em day trade, ou seja, tentar prever o preço daqui a 15 min. Acho que isso é bem mais difícil... Colocar um prazo de 1-3 semanas talvez seja mais viável. Podemos discutir isso ai.

2. Encontrar alguns ativos fortemente correlacionados com o ativo de interesse. Ou seja, encontrar outras ações que historicamente tendem a acompanhar a oscilação de preço desse ativo alvo. Quando uma sobe, a outra sobe também. Quando uma cai, a outra cai tb, etc. A ideia é selecionar outras ações que tem forte correlação para alimentar o modelo.

3. Tentar incorporar dados de análise fundamentalista apenas do ativo de interesse. Algo sobre patrimônio líquido, pagamento de dividendos, etc... - temos que ver quais dados seriam interessantes e o mais importante: como obtê-los kkk

4. Pegar isso tudo e alimentar um modelo de ML de regressão pra tentar prever o preço nas próximas 1-3 semanas. Avaliar a performance do modelo. É isso haha

In [ ]:
import numpy as np
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext Cython
import copt  # cython optimized stuff

import pandas as pd

df = pd.read_csv('data/raw_data.txt')
df.info()
df.head()

In [ ]:
# import cProfile
# cProfile.run('df[\'datahora\'].apply(lambda x: datetime.datetime.strptime(str(x), \'%Y%m%d%H%M\'))')
# cProfile.run('pd.to_datetime(df[\'datahora\'], format=\'%Y%m%d%H%M\')')
# cProfile.run('df[\'datahora\'].apply(lambda x: i64todt(x))')

In [ ]:
df['datahora'].apply(lambda x: copt.i64todt(x))

<hr></hr>

In [ ]:
# Pega apenas os dados de interesse, renomeia as colunas e coloca o datetime de índice
data = df[['codigo', 'fechamento_atual', 'volume_financeiro', 'datahora']]
data.columns = ['code', 'close', 'volume', 'datetime']
data.set_index('datetime', inplace=True)

In [ ]:
# Organiza os dados em .csv separado para cada empresa, já agrupados por dia

import os

companies = data.code.unique()

if not os.path.exists(r'C:\Users\aleex\Desktop\dados_financas\companies'):
    os.makedirs(r'C:\Users\aleex\Desktop\dados_financas\companies')
    
for companie in companies:
    if not os.path.exists(r'C:\Users\aleex\Desktop\dados_financas\companies\{}_1d.csv'.format(companie)):
        df_dumb = data[data['code'] == '{}'.format(companie)]
        df_dumb = df_dumb[['close', 'volume']]
        df_dumb_1d = df_dumb['close'].resample('1D').ohlc() # Transforma em dados diários - fechamento
        df_dumb_volume = df_dumb['volume'].resample('1D').sum() # Transforma em dados diários - volume
        df_dumb_1d = df_dumb_1d.join(df_dumb_volume, how='inner')
        df_dumb_1d.to_csv(r'C:\Users\aleex\Desktop\dados_financas\companies\{}_1d.csv'.format(companie))
        
    print(companie)

In [ ]:
# Abre cada arquivo que foi gerado, obtém os dados de fechamento de cada empresa e salva em um df único

main_df = pd.DataFrame()
    
for companie in companies:
    df = pd.read_csv(r'C:\Users\aleex\Desktop\dados_financas\companies\{}_1d.csv'.format(companie), index_col='datetime')
    df.rename(columns = {'close': companie}, inplace=True)
    df.drop(['open', 'high', 'low', 'volume'], axis = 1, inplace=True)
    
    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df, how='outer')
    
    print('Juntando dados da empresa {}...'.format(companie))
    
main_df.head()

In [ ]:
# Faz um dataframe com a correlações e plota o heatmap de parte do df de correlações

df_corr = main_df.pct_change(periods=7) # Diferença percentual semanal dos preços
df_corr = df_corr.corr()
print(df_corr)

correlations = df_corr.values

fig, ax = plt.subplots(figsize=(20, 20))
heatmap = ax.pcolor(correlations[0:11,0:11], cmap = 'RdYlGn')
fig.colorbar(heatmap)
ax.set_xticks(np.arange(correlations[0:11,0:11].shape[0]) + 0.5, minor=False)
ax.set_yticks(np.arange(correlations[0:11,0:11].shape[1]) + 0.5, minor=False)
ax.invert_yaxis()
ax.xaxis.tick_top()

ax.set_xticklabels(df_corr[0:11].columns)
ax.set_yticklabels(df_corr[0:11].index)
ax.set_title('Correlations', pad= 30, fontdict = {'fontsize': 20})
plt.show()

In [ ]:
# Conta quantas correlações maiores que 0.9 ou -0.9 existem para cada ativo
# É utilizado na escolha do ativo alvo

pd.options.display.max_rows = 999
df_boolean = ((df_corr > 0.7) | (df_corr < - 0.1))

companies = []
number_of_corr = []

for companie in df_corr.columns:
    sum_corr = np.sum(df_boolean[companie])
    companies.append(companie)
    number_of_corr.append(sum_corr)
    
df_sum_corr = pd.DataFrame(list(zip(companies, number_of_corr)), columns=['Companie', 'Number_of_correlations'])
df_sum_corr

In [ ]:
# Reparei que a CSN tem 4 ações com forte correlação. Todas do setor metalúrgico, menos a BRAP4.
# Pegar ela de exemplo aqui (pdoemos escolher outro ativo se quiser)

df_csn = df_corr['CSNA3'][(df_corr['CSNA3'] > 0.7) | (df_corr['CSNA3'] < - 0.1)]
df_csn

In [ ]:
# Pego o histórico de preços só das ações que selecionei pela correlação

df_csn_corr = main_df[['BRAP4', 'GGBR4', 'USIM5', 'VALE3', 'CSNA3']]
df_csn_corr = df_csn_corr.dropna()
df_csn_corr.head()

In [ ]:
# Ploto um scatter matrix pra visualização

from pandas.plotting import scatter_matrix

scatter_matrix(df_csn_corr, figsize=(20,20), alpha=0.6)
plt.show()

Mano, da pra ver que existem correlação bem fortes... por exemplo entre VALE3 e BRAP4...

Aparentemente não existem correlações negativas tão fortes quanto existem correlações positivas entre as empresas... Não sei pq

A ideia é selecionar alguns ativos fortemente correlacionados com o que a gente escolher para alimentar o modelo

In [ ]:
date = pd.to_datetime(df_csn_corr.index, format = '%Y-%m-%d') # Só transforma pra datetime para conseguir plotar

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(date, df_csn_corr['VALE3'], label = 'VALE3')
ax.plot(date, df_csn_corr['CSNA3'], label = 'CSNA3')
ax.legend(loc = 2, prop={'size': 15})


ax.set_ylabel('Preço do papel')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(date, df_csn_corr['CSNA3'], label = 'CSNA3')
ax.plot(date, df_csn_corr['USIM5'], label = 'USIM5')
ax.legend(loc = 2, prop={'size': 15})


ax.set_ylabel('Preço do papel')

plt.show()

Tipo, da pra ver que de fato há uma relação forte entre a variação de preço dessas empresas... mas a minha impressão pelo menos é que essa relação era mais forte no passado e foi se perdendo um pouco a partir de 2016 assim. Tem que ver isso também né... as vezes é melhor a gente olhar para as correlações calculadas só em anos mais recentes... tipo de 2016 pra frente.

Esse é outro ponto para discutirmos também...